# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 17 2022 2:07PM,244396,20,8107365-FLS-BO-FLS,"ACI Healthcare USA, Inc.",Released
1,Aug 17 2022 2:07PM,244396,20,8201063-BO,"ACI Healthcare USA, Inc.",Released
2,Aug 17 2022 2:07PM,244396,20,8244109,"ACI Healthcare USA, Inc.",Released
3,Aug 17 2022 2:07PM,244396,20,8259925,"ACI Healthcare USA, Inc.",Released
4,Aug 17 2022 2:07PM,244396,20,8259936,"ACI Healthcare USA, Inc.",Released
5,Aug 17 2022 2:07PM,244396,20,8259935-BO,"ACI Healthcare USA, Inc.",Released
6,Aug 17 2022 2:07PM,244396,20,8259923-BO,"ACI Healthcare USA, Inc.",Released
7,Aug 17 2022 2:07PM,244396,20,8259928-BO,"ACI Healthcare USA, Inc.",Released
8,Aug 17 2022 2:07PM,244396,20,8259929-BO,"ACI Healthcare USA, Inc.",Released
9,Aug 17 2022 2:07PM,244396,20,8208667-BO1,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,244393,Executing,1
52,244394,Released,1
53,244395,Released,1
54,244396,Released,73
55,244397,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244393,NaN,1.0,NaN
244394,NaN,NaN,1.0
244395,NaN,NaN,1.0
244396,NaN,NaN,73.0
244397,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244307,0.0,0.0,5.0
244312,0.0,5.0,16.0
244313,0.0,0.0,2.0
244315,0.0,0.0,1.0
244316,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244307,0,0,5
244312,0,5,16
244313,0,0,2
244315,0,0,1
244316,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244307,0,0,5
1,244312,0,5,16
2,244313,0,0,2
3,244315,0,0,1
4,244316,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244307,,,5
1,244312,,5,16
2,244313,,,2
3,244315,,,1
4,244316,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 17 2022 2:07PM,244396,20,"ACI Healthcare USA, Inc."
73,Aug 17 2022 2:04PM,244397,19,"Emersa Waterbox, LLC"
74,Aug 17 2022 1:53PM,244395,10,Yusen – 3D Matrix
75,Aug 17 2022 1:48PM,244394,10,Yusen – 3D Matrix
76,Aug 17 2022 1:44PM,244393,20,"HVL, LLC dba Atrium Innovations"
77,Aug 17 2022 1:43PM,244392,10,ISDIN Corporation
87,Aug 17 2022 1:36PM,244368,20,Alumier Labs Inc.
93,Aug 17 2022 1:34PM,244389,10,"ACI Healthcare USA, Inc."
94,Aug 17 2022 1:34PM,244389,10,ISDIN Corporation
147,Aug 17 2022 1:28PM,244391,12,SugarBear


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 17 2022 2:07PM,244396,20,"ACI Healthcare USA, Inc.",,,73
1,Aug 17 2022 2:04PM,244397,19,"Emersa Waterbox, LLC",,,1
2,Aug 17 2022 1:53PM,244395,10,Yusen – 3D Matrix,,,1
3,Aug 17 2022 1:48PM,244394,10,Yusen – 3D Matrix,,,1
4,Aug 17 2022 1:44PM,244393,20,"HVL, LLC dba Atrium Innovations",,1,
5,Aug 17 2022 1:43PM,244392,10,ISDIN Corporation,,,10
6,Aug 17 2022 1:36PM,244368,20,Alumier Labs Inc.,,1,5
7,Aug 17 2022 1:34PM,244389,10,"ACI Healthcare USA, Inc.",,,54
8,Aug 17 2022 1:34PM,244389,10,ISDIN Corporation,,,54
9,Aug 17 2022 1:28PM,244391,12,SugarBear,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 2:07PM,244396,20,"ACI Healthcare USA, Inc.",73,,
1,Aug 17 2022 2:04PM,244397,19,"Emersa Waterbox, LLC",1,,
2,Aug 17 2022 1:53PM,244395,10,Yusen – 3D Matrix,1,,
3,Aug 17 2022 1:48PM,244394,10,Yusen – 3D Matrix,1,,
4,Aug 17 2022 1:44PM,244393,20,"HVL, LLC dba Atrium Innovations",,1,
5,Aug 17 2022 1:43PM,244392,10,ISDIN Corporation,10,,
6,Aug 17 2022 1:36PM,244368,20,Alumier Labs Inc.,5,1,
7,Aug 17 2022 1:34PM,244389,10,"ACI Healthcare USA, Inc.",54,,
8,Aug 17 2022 1:34PM,244389,10,ISDIN Corporation,54,,
9,Aug 17 2022 1:28PM,244391,12,SugarBear,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 2:07PM,244396,20,"ACI Healthcare USA, Inc.",73,,
1,Aug 17 2022 2:04PM,244397,19,"Emersa Waterbox, LLC",1,,
2,Aug 17 2022 1:53PM,244395,10,Yusen – 3D Matrix,1,,
3,Aug 17 2022 1:48PM,244394,10,Yusen – 3D Matrix,1,,
4,Aug 17 2022 1:44PM,244393,20,"HVL, LLC dba Atrium Innovations",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 2:07PM,244396,20,"ACI Healthcare USA, Inc.",73.0,NaN,NaN
1,Aug 17 2022 2:04PM,244397,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Aug 17 2022 1:53PM,244395,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Aug 17 2022 1:48PM,244394,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Aug 17 2022 1:44PM,244393,20,"HVL, LLC dba Atrium Innovations",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 2:07PM,244396,20,"ACI Healthcare USA, Inc.",73.0,0.0,0.0
1,Aug 17 2022 2:04PM,244397,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Aug 17 2022 1:53PM,244395,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Aug 17 2022 1:48PM,244394,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Aug 17 2022 1:44PM,244393,20,"HVL, LLC dba Atrium Innovations",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5620226,195.0,25.0,0.0
102,244320,1.0,0.0,0.0
12,488781,2.0,0.0,0.0
15,733074,5.0,16.0,10.0
17,488643,2.0,0.0,0.0
19,488782,9.0,7.0,0.0
20,1466145,120.0,5.0,0.0
21,733003,3.0,0.0,0.0
22,733133,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5620226,195.0,25.0,0.0
1,102,244320,1.0,0.0,0.0
2,12,488781,2.0,0.0,0.0
3,15,733074,5.0,16.0,10.0
4,17,488643,2.0,0.0,0.0
5,19,488782,9.0,7.0,0.0
6,20,1466145,120.0,5.0,0.0
7,21,733003,3.0,0.0,0.0
8,22,733133,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,195.0,25.0,0.0
1,102,1.0,0.0,0.0
2,12,2.0,0.0,0.0
3,15,5.0,16.0,10.0
4,17,2.0,0.0,0.0
5,19,9.0,7.0,0.0
6,20,120.0,5.0,0.0
7,21,3.0,0.0,0.0
8,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,195.0
1,102,Released,1.0
2,12,Released,2.0
3,15,Released,5.0
4,17,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,17,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
Executing,25.0,0.0,0.0,16.0,0.0,7.0,5.0,0.0,0.0
Released,195.0,1.0,2.0,5.0,2.0,9.0,120.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,17,19,20,21,22
0,Completed,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,25.0,0.0,0.0,16.0,0.0,7.0,5.0,0.0,0.0
2,Released,195.0,1.0,2.0,5.0,2.0,9.0,120.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,17,19,20,21,22
0,Completed,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,25.0,0.0,0.0,16.0,0.0,7.0,5.0,0.0,0.0
2,Released,195.0,1.0,2.0,5.0,2.0,9.0,120.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()